In [1]:
import os
import glob
import flywheel
import logging
import zipfile
import platform

In [2]:
#Initialize the logger
logging.basicConfig(filename='download_mrs.log', level=logging.INFO)
logger = logging.getLogger()

In [3]:
# Get API Key from local environment variables and initialize the Flywheel client
#api_key = os.environ.get('API_KEY')
fw = flywheel.Client("cuny-mri.flywheel.io:djEhbcLgXzMrzELz7XHKFCXfZVSsl9cYDDRFs9Hhue6mjBowm8o_0J06Q")

In [4]:
# Path where the MRS data will be downloaded
root_path = '/Users/jacekdmochowski/PROJECTS/fus/data/mrs_flywheel/'

# Get the Flywheel Project
project = fw.lookup('jacek/FusRelabeledCopy')

In [5]:
if platform.system == 'Windows':
    print('w')
    #root_path = 'C:\\Users\\jdm82\\Desktop\\fus\\data\\mrs_flywheel'
else:
    print('l')
    #root_path = '/Users/jacekdmochowski/PROJECTS/fus/data/mrs_flywheel'

l


In [6]:
def download_all_mrs(root_data):
    #Get the group label
    group_label = project.group
    print(group_label)
    
    #Iterate through all the files in the 'ACTIVE' and 'SHAM' sessions in the project
    for subject in project.subjects():
        
        for session in subject.sessions():
            
            # Input the session labels with MRS data.
            
            if session.label == 'ACTIVE' or session.label == 'SHAM': 
                
                for acquisition in session.acquisitions():
                    
                    acq_label = acquisition.label
                    
                    #Check if the acquisition label starts with 'mrs'
                    
                    if acq_label.startswith('mrs'):
                        
                        #Create the directory using the Flywheel hierarchy
                        if platform.system == 'Windows':
                            path = f'{root_path}\\{group_label}\\{project.label}\\{subject.label}\\{session.label}\\{acq_label}'
                        else:
                            path = f'{root_path}/{group_label}/{project.label}/{subject.label}/{session.label}/{acq_label}'

                        #path = f'{root_path}\\{group_label}\\{project.label}\\{subject.label}\\{session.label}\\{acq_label}'
                        os.makedirs(path, exist_ok=True)
                        
                        for file in acquisition.files:
                            
                            #Find the dicom file in the mrs acquisition
                            if file.type == 'dicom':

                                if platform.system == 'Windows':
                                    file_path = f'{path}\\{file.name}'
                                else:
                                    file_path = f'{path}/{file.name}'
                                        
                                file_name = f'sub-{subject.label}_ses-{session.label}_acq-{acq_label}'

                                if platform.system == 'Windows':
                                    full_file_name = f'{path}\\{file_name}'
                                else:
                                    full_file_name = f'{path}/{file_name}'

                                if not os.path.exists(full_file_name):
                                    
                                    try:
                                        #Download the file from the acquisition.
                                        
                                        acquisition.download_file(file.name, file_path)
                                        
                                        # Open the zip file
                                        with zipfile.ZipFile(file_path, 'r') as zip_ref:
                                            #Extract the contents of the zip file to the path
                                            zip_ref.extractall(path)
                                            
                                        #Delete compressed zip file
                                        os.remove(file_path)
                                        
                                        #Rename the unzipped folder to the new file name
                                        if platform.system == 'Windows':
                                            unzipped_folder = glob.glob(f'{path}\\*')[0]
                                            os.rename(unzipped_folder, f'{path}\\{file_name}')
                                        else:
                                            unzipped_folder = glob.glob(f'{path}/*')[0]
                                            os.rename(unzipped_folder, f'{path}/{file_name}')

                                        logger.info(f'Downloaded {file.name} from {acq_label} in {session.label} for {subject.label}')
                                        
                                        print(f'Downloaded {file.name} from {acq_label} in {session.label} for {subject.label}')
                                        
                                    except Exception as e:
                                        logger.error(f'Error downloading {file.name} from {acq_label} in {session.label} for {subject.label}. {e}')
                                break


In [ ]:
download_all_mrs(root_path)

jacek
Downloaded 1.3.12.2.1107.5.2.43.166037.2024061716491952796094588.0.0.0.dicom.zip from mrs-svs_task-post in ACTIVE for MATHAI
Downloaded 1.3.12.2.1107.5.2.43.166037.2024061716272511796997577.0.0.0.dicom.zip from mrs-svs_task-pre in ACTIVE for MATHAI
Downloaded 1.3.12.2.1107.5.2.43.166037.2024070315524590556268552.0.0.0.dicom.zip from mrs-svs_task-post in SHAM for MATHAI
Downloaded 1.3.12.2.1107.5.2.43.166037.2024070315322350952271541.0.0.0.dicom.zip from mrs-svs_task-pre in SHAM for MATHAI
Downloaded 1.3.12.2.1107.5.2.43.166037.2024061717332148866820706.0.0.0.dicom.zip from mrs-svs_task-pre in SHAM for LIN
Downloaded 1.3.12.2.1107.5.2.43.166037.2024061717540499185717717.0.0.0.dicom.zip from mrs-svs_task-post in SHAM for LIN
Downloaded 1.3.12.2.1107.5.2.43.166037.2024070316195391242993941.0.0.0.dicom.zip from mrs-svs_task-pre in ACTIVE for LIN
Downloaded 1.3.12.2.1107.5.2.43.166037.202407031640216879390958.0.0.0.dicom.zip from mrs-svs_task-post in ACTIVE for LIN
Downloaded 1.3.12.2